# Установка библиотек

In [ ]:
# Установка необходимых библиотек
!pip install arize-phoenix openinference-instrumentation-mistralai mistralai pymupdf nltk

# Импорт библиотек

In [ ]:
# Импортируем библиотеки
import phoenix as px
from phoenix.otel import register
import os
from mistralai import Mistral
from mistralai.models import UserMessage
import fitz  # PyMuPDF для работы с PDF
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Ресурсы для отчистки текста

In [ ]:
# Скачиваем необходимые ресурсы
nltk.download('punkt_tab')
nltk.download('stopwords')

# Запуск интерфейса Phoenix

In [ ]:
# Инициализация Phoenix
px.launch_app()

In [ ]:
# Регистрация трассировщика для Phoenix
tracer_provider = register(
  project_name="my-llm-app",  # Можно оставить как 'default'
)

# Используем mistral для генерации ответов

In [ ]:
# Устанавливаем API-ключ для Mistral (в Google Colab вы можете использовать переменные окружения)
# Введите ваш ключ в следующей строке
os.environ["MISTRAL_API_KEY"] = "PglyBm40sXxMmjd2jFvRaOgoZAh1Nb0g" #Ключ одноразовый, для работы замените на свой

In [ ]:
# Создание клиента Mistral
api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-tiny"

client = Mistral(api_key=api_key)

# Загрузка книги "Алиса в стране чудес"

In [ ]:
# Скачиваем книгу "Алиса в стране чудес"
!wget https://storage.yandexcloud.net/academy.ai/book/Alices-Adventures-in-Wonderland-by-Lewis-Carroll-PDF-Book.pdf

# Предобработка книги для уменьшения вероятности галлюцинации

In [ ]:
# Открываем PDF файл
pdf_path = "Alices-Adventures-in-Wonderland-by-Lewis-Carroll-PDF-Book.pdf"
doc = fitz.open(pdf_path)

In [ ]:
# Извлекаем текст из всех страниц
raw_text = ""
for page_num in range(doc.page_count):
    page = doc.load_page(page_num)
    raw_text += page.get_text()

In [ ]:
# Очистка текста: удаление специальных символов, стоп-слов
def clean_text(text):
    # Удаление специальных символов и чисел
    text = re.sub(r'[^A-Za-z\s]', '', text)

    # Преобразуем в нижний регистр
    text = text.lower()

    # Токенизация текста
    tokens = word_tokenize(text)

    # Удаляем стоп-слова
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Воссоздаем текст
    cleaned_text = ' '.join(filtered_tokens)
    return cleaned_text

In [ ]:
# Очищаем текст перед подачей в модель
cleaned_text = clean_text(raw_text)

# Запрос к модели

In [ ]:
# Пример промпта с указанием на признание неопределенности
prompt = f"Please summarize the following text. If you are unsure about something, please say 'I don't know'. {cleaned_text[:1000]}"

In [ ]:
# Отправка запроса модели
chat_response = client.chat.complete(
    model=model,
    messages=[UserMessage(content=prompt)]
)

# Печатаем ответ модели
print(chat_response.choices[0].message.content)

# Итог

Интерфейс Phoenix показывает следующие результаты:

## LLM Input

```json
{
  "model": "mistral-tiny",
  "messages": [
    {
      "content": "Please summarize the following text. If you are unsure about something, please say 'I don't know'. alices adventures wonderland lewis carroll project gutenberg ebook alices adventures wonderland ebook use anyone anywhere united states parts world cost almost restrictions whatsoever may copy give away reuse terms project gutenberg license included ebook online wwwgutenbergorg located united states check laws country located using ebook hope enjoyed reading come back httpslearnenglishnewcom find fascinating exciting stories httpslearnenglishnewcom alices adventures wonderland lewis carroll millennium fulcrum edition contents chapter rabbithole chapter ii pool tears chapter iii caucusrace long tale chapter iv rabbit sends little bill chapter v advice caterpillar chapter vi pig pepper chapter vii mad teaparty chapter viii queens croquetground chapter ix mock turtles story chapter x lobster quadrille chapter xi stole tarts chapter xii alices evidence chapter rabbithole alice beginning get tired sitting sister bank nothing twice peeped book sister reading pictures conversations use book t",
      "role": "user"
    }
  ],
  "temperature": "",
  "max_tokens": "",
  "stream": false,
  "random_seed": "",
  "tools": "",
  "n": "",
  "retries": ""
}
```

## assistant

```json
The text is about the Project Gutenberg eBook of "Alice's Adventures in Wonderland" by Lewis Carroll. The eBook is free and can be copied, given away, reused, and shared with no restrictions whatsoever. It can be found online at projectgutenberg.org. The text also mentions that the laws in different countries may apply when using the eBook. The Millennium Fulcrum edition of the book has various chapters, such as "The Rabbit Hole," "Pool of Tears," "The Caucus Race," and "The Mad Tea-Party." Alice, the protagonist, begins her adventure when she gets tired of sitting by the bank and looking at her sister reading pictures in the book. She peeps at the book and starts talking to the characters in the conversations.
```

Вывод: Модель обработала текст из книги "Алиса в Стране Чудес" Льюиса Кэрролла, предоставленный в запросе, который является частью лицензированного проекта Project Gutenberg. Текст описывает различные главы книги, начиная с того, как Алиса попадает в кроличью нору, и заканчивая такими ключевыми моментами, как её участие в безумной чайной вечеринке и суде.